In [ ]:
#### LIBRERIAS #### 

import pandas as pd
import os, sys
import datetime
from datetime import date
from pathlib import Path 
from datetime import datetime
import openpyxl 
import warnings
warnings.filterwarnings('ignore')
import win32api
import win32com.client
import os, sys


#### ----------------------------- ####
 
today = datetime.today().strftime('%d-%m-%y')
path = r'\\vspmdhq-bi\BI_Server_F\BI_for_Reservations\GSO\Meetings_SQL_Database\01_Inputs'
files = os.listdir(path)

#Creamos el Set de unique_endings y la lista de files_STLY

unique_endings_STLY = set()
files_STLY = []

# #CONTAMOS EL NÚMERO DE ARCHIVOS STLY
for i in files:
    if "STLY_" in i:
        files_STLY.append(i)

# #EXTRAEMOS EL PESO DE CADA ARCHIVO STLY
file_STLY_Size = {}
for file in files_STLY:
    file_path = os.path.join(path,file)
    size_bytes = os.path.getsize(file_path)
    size_kb = size_bytes/1024
    size_mb = size_kb/1024
    file_STLY_Size[file] = {
        "Archivo": file,
        "Size (Bytes)": size_bytes,
        "Size (KB)": size_kb,
        "Size (MB)": size_mb
    }
    
#    #PROCESO PARA VER LOS NÚMEROS DE TERMINACIONES QUE HAY EN CADA ARCHIVO 
    try:
        df = pd.read_csv(file_path, delimiter=";")
        column_2 = df.iloc[:,1]

        for data in column_2:
            ending = str(data)[-2:]
            unique_endings_STLY.add((file,ending))
    except pd.errors.ParserError as e:
        print(f"Error al procesar el archivo: {file}")
        print(f"Detalles del error: {e}")
    
# # CREAMOS LOS DATAFRAME

df_unique_endings = pd.DataFrame(list(unique_endings_STLY),columns=['Archivo','Terminación'])

df_sizes_STLY = pd.DataFrame(file_STLY_Size).T

# #HACEMOS MERGE DE AMBOS

all_info_STLY = df_sizes_STLY.merge(df_unique_endings,left_on="Archivo", right_on="Archivo")
all_info_STLY.sort_values(by=["Terminación"],inplace=True)

print("Creada tabla all_info_STLY")

# REPETIMOS PROCESO CON ARCHIVOS DE TODAY #

#Creamos el Set de unique_endings y la lista de files_TODAY

unique_endings_TODAY = set()
files_TODAY = []

#CONTAMOS EL NÚMERO DE ARCHIVOS STLY
for i in files:
    if "TODAY_" in i:
        files_TODAY.append(i)

#EXTRAEMOS EL PESO DE CADA ARCHIVO STLY
file_TODAY_Size = {}
for file in files_TODAY:
    file_path = os.path.join(path,file)
    size_bytes = os.path.getsize(file_path)
    size_kb = size_bytes/1024
    size_mb = size_kb/1024
    file_TODAY_Size[file] = {
        "Archivo": file,
        "Size (Bytes)": size_bytes,
        "Size (KB)": size_kb,
        "Size (MB)": size_mb
    }
    
   #PROCESO PARA VER LOS NÚMEROS DE TERMINACIONES QUE HAY EN CADA ARCHIVO 
    try: 
        df = pd.read_csv(file_path, delimiter=";")
        column_2 = df.iloc[:,1]

        for data in column_2:
            ending = str(data)[-2:]
            unique_endings_TODAY.add((file,ending))
    except pd.errors.ParserError as e:
        print(f"Error al procesar el archivo: {file}")
        print(f"Detalles del error: {e}")



    
# CREAMOS LOS DATAFRAME

df_unique_endings_TODAY = pd.DataFrame(list(unique_endings_TODAY),columns=['Archivo','Terminación'])

df_sizes_TODAY = pd.DataFrame(file_TODAY_Size).T

#HACEMOS MERGE DE AMBOS

all_info_TODAY = df_sizes_TODAY.merge(df_unique_endings_TODAY,left_on="Archivo", right_on="Archivo")
all_info_TODAY.sort_values(by=["Terminación"],inplace=True)

print("Creada tabla all_info_Today")

# PATH PARA GUARDAR LOS ARCHIVOS
output_path_STLY  = fr'\\vspmdhq-bi\BI_Server_F\BI_for_Reservations\GSO\Meetings_SQL_Database\12_Checks\MEETINGS LOGS\STLY_Sizes_{today}.xlsx'
output_path_TODAY = fr'\\vspmdhq-bi\BI_Server_F\BI_for_Reservations\GSO\Meetings_SQL_Database\12_Checks\MEETINGS LOGS\TODAY_Sizes_{today}.xlsx'


all_info_STLY.to_excel(output_path_STLY)
all_info_TODAY.to_excel(output_path_TODAY)

#ENVIAR AL MAIL

user = os.getlogin()
outlook_mail = win32com.client.Dispatch("Outlook.Application")
mail = outlook_mail.CreateItem(0x0)
mail.Subject = 'Check Archivos Meetings '+ today
attachment = mail.Attachments.Add(output_path_STLY)
attachment2 = mail.Attachments.Add(output_path_TODAY)
# attachment = mail.Attachments.Add(fr"F:\BI_for_Reservations\GSO\Meetings_SQL_Database\12_Checks\MEETINGS LOGS\TODAY_Sizes_{today}.xlsx")
# attachment2 = mail.attachments.Add(fr"F:\BI_for_Reservations\GSO\Meetings_SQL_Database\12_Checks\MEETINGS LOGS\STLY_Sizes_{today}.xlsx")
# mail.To = 'ej.garcia@minor-hotels.com'
mail.To= 'ej.garcia@minor-hotels.com;j.mateu@minor-hotels.com'
mail.HTMLBody = f"""
    Hay {len(df_sizes_TODAY)} archivos TODAY <br> 
    <br> Y hay {len(df_sizes_STLY)} archivos de STLY
"""
mail.Send()

In [3]:
files_TODAY

['MT_BOMT_OD_TODAY_0.csv',
 'MT_BOMT_OD_TODAY_1.csv',
 'MT_BOMT_OD_TODAY_10.csv',
 'MT_BOMT_OD_TODAY_11.csv',
 'MT_BOMT_OD_TODAY_12.csv',
 'MT_BOMT_OD_TODAY_13.csv',
 'MT_BOMT_OD_TODAY_14.csv',
 'MT_BOMT_OD_TODAY_15.csv',
 'MT_BOMT_OD_TODAY_16.csv',
 'MT_BOMT_OD_TODAY_17.csv',
 'MT_BOMT_OD_TODAY_18.csv',
 'MT_BOMT_OD_TODAY_19.csv',
 'MT_BOMT_OD_TODAY_2.csv',
 'MT_BOMT_OD_TODAY_20.csv',
 'MT_BOMT_OD_TODAY_21.csv',
 'MT_BOMT_OD_TODAY_22.csv',
 'MT_BOMT_OD_TODAY_23.csv',
 'MT_BOMT_OD_TODAY_24.csv',
 'MT_BOMT_OD_TODAY_25.csv',
 'MT_BOMT_OD_TODAY_26.csv',
 'MT_BOMT_OD_TODAY_27.csv',
 'MT_BOMT_OD_TODAY_28.csv',
 'MT_BOMT_OD_TODAY_29.csv',
 'MT_BOMT_OD_TODAY_3.csv',
 'MT_BOMT_OD_TODAY_30.csv',
 'MT_BOMT_OD_TODAY_31.csv',
 'MT_BOMT_OD_TODAY_32.csv',
 'MT_BOMT_OD_TODAY_33.csv',
 'MT_BOMT_OD_TODAY_34.csv',
 'MT_BOMT_OD_TODAY_35.csv',
 'MT_BOMT_OD_TODAY_36.csv',
 'MT_BOMT_OD_TODAY_37.csv',
 'MT_BOMT_OD_TODAY_38.csv',
 'MT_BOMT_OD_TODAY_39.csv',
 'MT_BOMT_OD_TODAY_4.csv',
 'MT_BOMT_OD_TODAY_40.csv